In [2]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [3]:
available_functions = {}

In [12]:
avalible_tools = [{"type": "retrieval"}]

In [21]:
# A Class to Manage All Open API Assistant Calls and Functions
from openai.types.beta.threads import Run, ThreadMessage
from openai.types.beta.thread import Thread
from openai.types.beta.assistant_create_params import Tool

import time
import json

class PDFChatManager:
    def __init__(self, model: str = "gpt-3.5-turbo-1106"):
        self.client = OpenAI()
        self.model = model
        self.assistant = None
        self.thread = None
        self.run = None

    def create_file(self, file_path: str, purpose: str='assistants') -> None:
        with open(file_path, "rb") as file:
            file_obj = self.client.files.create(file=file, purpose=purpose)
            self.file_id = file_obj.id

    def create_assistant(self, name: str, instructions: str, tools: list[Tool]) -> None:
        self.assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            tools=tools,
            model=self.model,
            file_ids=[self.file_id]
        )

    def create_thread(self) -> Thread:
        self.thread = self.client.beta.threads.create()
        return self.thread

    def add_message_to_thread(self, role: str, content: str) -> None:
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role=role,
            content=content
        )

    def run_assistant(self, instructions: str) -> Run:
        self.run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions=instructions
        )
        return self.run

    def wait_for_completion(self, run: Run, thread: Thread) -> Run:

        while run.status in ["in_progress", "queued"]:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=self.run.id
            )
            print(f"Run is {run.status} ...")
            time.sleep(3)  # Wait for 3 seconds before checking again

            if run_status.status == 'completed':
                processed_response = self.process_messages()
                return processed_response
                # break
            elif run_status.status == 'requires_action':
                print("Function Calling ...")
                self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
            elif run.status == "failed":
                print("Run failed.")
                break
            else:
                print(f"Waiting for the Assistant to process...: {run.status}")

    def process_messages(self) -> list[ThreadMessage]:
        messages: list[ThreadMessage] = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        return messages

    def call_required_functions(self, required_actions: dict):
        tool_outputs = []

        for action in required_actions["tool_calls"]:
            function_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            print('function_name', function_name)
            print('function_arguments', arguments)

            if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**arguments)
                    tool_outputs.append({
                        "tool_call_id": action['id'],
                        "output": output,
                    })

            else:
                raise ValueError(f"Unknown function: {function_name}")

        print("Submitting outputs back to the Assistant...")
        self.client.beta.threads.runs.submit_tool_outputs(
            thread_id=self.thread.id,
            run_id=self.run.id,
            tool_outputs=tool_outputs
        )



In [22]:
# Show Messages and Plot Images in Financial Analysis If ANY

import requests
from IPython.display import Image, display


def download_and_save_image(file_id: str, save_path: str) -> None:
    """
    Downloads an image from OpenAI using its file ID and saves it to the specified path.

    Args:
    - file_id (str): The ID of the file to download.
    - save_path (str): The path where the image will be saved.

    Returns:
    - None
    """
    # Construct the URL to download the image
    download_url = f"https://api.openai.com/v1/files/{file_id}/content"

    # Perform the HTTP GET request to download the image
    response = requests.get(download_url, headers={"Authorization": f"Bearer {os.environ.get("OPENAI_API_KEY")}"})

    # Check if the request was successful
    if response.status_code == 200:
        # Write the image to the specified file
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image downloaded and saved to {save_path}")
    else:
        print(f"Failed to download image: HTTP Status Code {response.status_code}")


def pretty_print(messages: list[ThreadMessage]) -> None:
    print("# Messages")
    for message in messages.data:
        role_label = "User" if message.role == "user" else "Assistant"
        # Check the type of message content and handle accordingly
        for content in message.content:
            if content.type == "text":
                message_content = content.text.value
                print(f"{role_label}: {message_content}\n")
                print()
            elif content.type == "image_file":
                # Handle image file content, e.g., print the file ID or download the image
                image_file_id = content.image_file.file_id
                # Define a path to save the image
                image_save_path = f"image_{image_file_id}.png"
                # Download and save the image
                print(f"{role_label}: Image file ID: {image_file_id}")
                download_and_save_image(image_file_id, image_save_path)

                # Display the image within Jupyter Notebook
                display(Image(filename=image_save_path))

In [23]:
def chat_with_pdf(file_path: str, assistant_intructions: str, question_to_ask: str) -> ThreadMessage:
    pdf_assistant : PDFChatManager = PDFChatManager()

    # 00 Create a file
    pdf_assistant.create_file(file_path=file_path)

    # 01 Create an assistant
    pdf_assistant.create_assistant(name="PDF Assistant", instructions=assistant_intructions, tools=avalible_tools)

    # 02 Create a thread
    pdf_assistant.create_thread()

    # 03 Add a message to the thread
    pdf_assistant.add_message_to_thread(role="user", content=question_to_ask)

    # 04 Run the assistant
    run = pdf_assistant.run_assistant(instructions="")

    # 05 Wait for the assistant to complete
    messages = pdf_assistant.wait_for_completion(run=run, thread=pdf_assistant.thread)

    #06 return response to be displayed
    return messages

    

In [24]:
ASSISTANT_SEED_PROMPT =  """ 

You are a specialized AI Assistant who efficiently manages and extracts information from PDF documents. 

Your role is to assist a diverse range of users, from business professionals to individuals, in navigating and understanding their PDF files. When interacting with users:

1. Identify the User's Objective from their Query

2. Request Specific Details: Encourage users to be specific about their needs. For instance, if they want to extract data, ask them to define the type of data (like dates, names, financial figures).

3. Understand the Context: Inquire about the nature of the document (e.g., financial report, academic paper) to tailor your assistance accordingly.

4. Communicate Clearly: Use straightforward, easy-to-understand language in your responses. Avoid technical jargon unless the user is comfortable with it.

5. Logical Question Sequencing: If a task requires multiple steps, guide the user through them in a logical order. For example, start with general extraction before moving to specific data points.

6. Prepare for Diverse Responses: Be ready to handle a range of user queries and rephrase your questions or guidance based on user feedback.

7. Iterate Based on User Feedback: If the user's response indicates misunderstanding, rephrase your guidance or provide additional clarifications.

Remember, your goal is to make the user's interaction with their PDFs more efficient and productive, respecting their privacy and time constraints. Don't say I don't know, instead, return you understanding or ask for more information.

"""

In [27]:
get_user_input = input("What is your question? ")

In [28]:
print(get_user_input)

Share Niches Impacts Discussed in Ch 2. And brainstorm 3 startup ideas and their details based on the niches discussed in ch 2


In [31]:
# Using Assistant

answer = chat_with_pdf(file_path="the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf", assistant_intructions=ASSISTANT_SEED_PROMPT, question_to_ask=get_user_input)

Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...


In [34]:
pretty_print(answer)

# Messages
Assistant: In Chapter 2 of the document "The Economic Potential of Generative AI: The Next Productivity Frontier", there is an extensive discussion about the transformation of customer operations through generative AI. The report highlights the potential of generative AI to revolutionize the entire customer operations function, improving customer experience and agent productivity through digital self-service and enhancing and augmenting agent skills. The technology has already demonstrated the ability to automate interactions with customers using natural language, resulting in improved issue resolution and reduced handling time while also diminishing agent attrition and requests to speak to managers. Specifically, the document discusses customer self-service interactions, customer-agent interactions, and agent self-improvement, outlining how generative AI can positively impact each of these areas【11†source】.

Based on this analysis, the following startup ideas can be brainst